### v1.0 : This notebook is a modified base reproduction an the existing notebook

## Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk
import os

2024-05-28 23:36:51.689998: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-28 23:36:51.690190: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-28 23:36:51.692237: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-28 23:36:51.716539: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 23:36:52.306830: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

### Global Variables

In [18]:
# Please ensure the files are available in the following locations
directory_data = "data"
directory_model = "models"
h_to_tau_tau_file = "hToTauTau_13TeV_PU20_filtered.h5"
background_file = "background_for_training.h5"
A_to_4_l_file = "Ato4l_lepFilter_13TeV_dataset.h5"
model_name = "40MHZ_norm_DNN.keras"
hChToTauNu_13TeV_PU20_file = "hChToTauNu_13TeV_PU20.h5"
leptoquark_LOWMASS_lepFilter_file= "leptoquark_LOWMASS_lepFilter_13TeV.h5"

## Global Functions

In [19]:
def create_AE(input_dim, h_dim_1, h_dim_2, latent_dim):
    # Encoder
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(h_dim_1, activation='relu')(inputs)
    x = layers.Dense(h_dim_2, activation='relu')(x)
    z = layers.Dense(latent_dim, activation='relu')(x)

    # Decoder
    x = layers.Dense(h_dim_2, activation='relu')(z)
    x = layers.Dense(h_dim_1, activation='relu')(x)
    outputs = layers.Dense(input_dim)(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

def loss_fn(y_true, y_pred):
    """masked mse"""
    mask = K.cast(K.not_equal(y_true, 0), K.floatx())
    squared_difference = K.square(mask * (y_pred - y_true))
    return K.mean(squared_difference)

def mse_loss(true, prediction):
    loss = np.mean(np.square(true - prediction), axis=-1)
    return loss

def AD_score(y, x):
    # masked mse
    mask = (y != 0)
    _x = x * mask
    _y = y * mask
    return (mse_loss(_y, _x))

def calculate_sensitivity(self,br_rate):
    AD=self.AD_cutoff
    sensitivity=[]
    for i,losses in enumerate(self.signal_loss):
        N=len(losses)
        n=0
        for loss in losses:
            if loss>=AD:
                n+=1
        sen=n/N
        sensitivity+=[sen]
    self.signal_sensitivity=sensitivity
    print(self.signal_sensitivity)


def list_datasets(file, path='/'):
    print(file[path])
    for key in file[path]:
        item = file[path + key]
        if isinstance(item, h5py.Dataset):
            # Print dataset path and shape (which describes its size/dimensions) (Datsets are columns here)
            print(f'Dataset: {path + key}, Shape: {item.shape}, Type: {item.dtype}')
        elif isinstance(item, h5py.Group):
            # It's a group, so recurse into it
            list_datasets(file, path + key + '/')

### Evaluators

In [5]:
class Model_Evaluator():
  def __init__(self,model_path,backround,signal,title='placeholder',save=False,labels=None):
    custom_objects = {'loss_fn': loss_fn}
    self.model = load_model(model_path, custom_objects=custom_objects)
    self.signal=signal
    self.backround=backround
    self.br_loss=[]
    self.signal_loss=[]
    self.backround_outputs=[]
    self.signal_outputs=[]
    self.title=title
    self.saveplots=save
    self.labels=labels

  def calculate_loss(self,batch_size):
    br=self.backround
    self.backround_outputs=self.model.predict(br)
    self.br_loss=AD_score(self.backround,self.backround_outputs)
    for i, batch in enumerate(self.signal):
      sr=batch
      self.signal_outputs+=[self.model.predict(sr)]
      self.signal_loss+=[AD_score(batch,self.signal_outputs[i])]
    return [self.br_loss,self.signal_loss]


  def histogram(self,bins):
    plt.hist(self.br_loss,bins=bins,histtype='step',label='backround num_events:{}'.format(len(self.br_loss)))
    for i,batch in enumerate(self.signal_loss):
      plt.hist(batch,bins=bins,histtype='step',label=str(self.labels[i+1])+" num_events:{}".format(len(batch)))
    plt.xlabel('loss')
    plt.ylabel('Frequency')
    plt.yscale('log')
    plt.title("{}_Hist".format(self.title))
    plt.legend()
    if self.saveplots==True:
      plt.savefig("{}_Hist.png".format(self.title), format="png", bbox_inches="tight")
    plt.show()

  def ROC(self):
    plt.plot(np.linspace(0,1,1000),np.linspace(0,1,1000),'--',label='diagonal')
    for j, batch in enumerate(self.signal_loss):
      truth=[]
      for i in range(len(self.br_loss)):
        truth+=[0]
      for i in range(len(batch)):
        truth+=[1]
      ROC_data=np.concatenate((self.br_loss,batch))
      fpr,tpr,x=sk.roc_curve(truth,ROC_data)
      auc=sk.roc_auc_score(truth,ROC_data)
      plt.plot(fpr,tpr,label=self.labels[j+1]+": "+str(auc))

    plt.xlabel('fpr')
    plt.semilogx()
    plt.ylabel('trp')
    plt.semilogy()
    plt.title("{}_ROC".format(self.title))
    plt.legend()
    if self.saveplots==True:
      plt.savefig("{}_ROC.png".format(self.title), format="png", bbox_inches="tight")
    plt.show()
  
  def Find_AD_Cutoff(self,br_rate,desired_rate,starting_AD):
    N=self.backround.shape[0]
    AD_max=starting_AD
    AD_List=np.linspace(0,AD_max,num=1000)
    best_AD=0
    for i,AD in enumerate(np.flip(AD_List)):
      n=0
      for loss in self.br_loss:
        if loss>=AD:
          n+=1
      sigrate=br_rate*n/N
      if sigrate<=desired_rate:
        best_AD=AD
      if sigrate>desired_rate:
        break
    self.AD_cutoff=best_AD
    return best_AD

### Set up Training Data Framework

In [6]:
f=h5py.File(os.path.join(directory_data, background_file), 'r')
Dataset=np.array(f["Particles"])

truthtable=[]

threshold=50
for i, batch in enumerate(Dataset):
  if np.sum(batch[:,0])>=threshold:
    truthtable+=[1]
  else:
    truthtable+=[0]

event_pt_br=[]
Data_Test_full=Dataset[2000001:3600000,:,:]
for j, br_1 in enumerate(Data_Test_full):
  event_pt_br+=[np.sum(br_1[:,0])]

for i, batch in enumerate(Dataset):
  pt_sum=0
  for j, particle in enumerate(Dataset[i,:,:]):
    if particle[3]!=0:
      pt_sum+=particle[0]
  for j, particle in enumerate(Dataset[i,:,:]):
    particle[0]=particle[0]/pt_sum

Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Test_Truth=truthtable[2000001:3600000]
Data_Validate=Dataset[3600001:4000000,:,0:3]

Data_Test_Flat=np.reshape(Data_Test,(-1,57))

### Populate Data

In [21]:
h_to_Tau_Tau=h5py.File(os.path.join(directory_data, h_to_tau_tau_file), 'r')
A_to_4_l=h5py.File(os.path.join(directory_data, A_to_4_l_file), 'r')
hC_to_Tau_Nu=h5py.File(os.path.join(directory_data, hChToTauNu_13TeV_PU20_file), 'r')
lepto=h5py.File(os.path.join(directory_data,leptoquark_LOWMASS_lepFilter_file), 'r')

h_tt_set=np.array(h_to_Tau_Tau["Particles"])
hC_tn_set=np.array(hC_to_Tau_Nu["Particles"])
A_4l_set=np.array(A_to_4_l["Particles"])
lepto_set=np.array(lepto["Particles"])
sets=[h_tt_set,hC_tn_set, A_4l_set,lepto_set]

for k, subset in enumerate(sets):
    for i, batch in enumerate(subset):
        pt_sum=0
    for j, particle in enumerate(subset[i,:,:]):
        if particle[3]!=0:
            pt_sum+=particle[0]
    for j, particle in enumerate(subset[i,:,:]):
        particle[0]=particle[0]/pt_sum
    print('one set done')

normed_signals=[]
for j, subset in enumerate(sets):
    normed_signals+=[np.reshape(subset[:,:,0:3],(-1,57))]

sig_label=['Backround','hC_tn','h_tt','A_4l','leptoquark']

one set done
one set done
one set done
one set done


### Look at File Data

In [22]:
with h5py.File(os.path.join(directory_data, A_to_4_l_file), 'r') as file:
    list_datasets(file)

<HDF5 group "/" (3 members)>
Dataset: /Particles, Shape: (55969, 19, 4), Type: float64
Dataset: /Particles_Classes, Shape: (4,), Type: |S16
Dataset: /Particles_Names, Shape: (4,), Type: |S5


### Evaluation

In [ ]:
evaluation=Model_Evaluator('models/40MHZ_norm_DNN.keras', Data_Test_Flat, normed_signals, title='Normalized 40MHZ DNN AE', save=True, labels=sig_label)
evaluation.calculate_loss(1024)
evaluation.histogram(bins=100)
evaluation.ROC()

21414/50000 ━━━━━━━━━━━━━━━━━━━━ 10s 359us/step